In [32]:
from bs4 import BeautifulSoup, SoupStrainer
import httplib2
import re
import pandas as pd
from itertools import chain
import time 

In [33]:
scp_main_df = pd.DataFrame(columns=['code', 'title', 'text', 'link', 'rating'], index=None)

In [34]:
http = httplib2.Http()

In [ ]:
status, response = http.request('https://scp-wiki.wikidot.com/scp-series')
soup = BeautifulSoup(response, 'html.parser')

# the page links all follow pattern and can loop through nums to string just zero padded basically
for i in range(1,7000):
  # timeout to limit load on the site
  if i % 240 == 0:
    print(i)
    time.sleep(60)
  num_as_str = '{}{}'.format('0' * (3 - len(str(i))) , str(i))
  url = 'https://scp-wiki.wikidot.com/scp-{}'.format(num_as_str)
  status, response = http.request(url)
  soup = BeautifulSoup(response, 'html.parser')
  text = [i for i in soup.find(id = 'page-content').find_all('p')]
  # p tags include things like image captions may want to exclude, can do with scp-image-caption class there and check other cases
  for i in text:
    if hasattr(i.parent, 'class'):
      if i.parent.get("class") == 'scp-image-caption':
        text.remove(i)
  text = [i.text for i in text]
  rating = soup.find(class_ = 'number prw54353')
  if rating == None:
    pass
  else:
    rating = rating.text
  
  status, response = http.request('https://scp-wiki.wikidot.com/scp-series')
  soup = BeautifulSoup(response, 'html.parser')
  title = soup.find('a', text='SCP-{}'.format(num_as_str)).parent.text.split(" - ")[-1]
  
  # print(rating)
  # quotes to preserve commas and not cause csv issues
  info = ['SCP-{}'.format(num_as_str), "\"{}\"".format(title), "\"{}\"".format(text), url, rating]
  scp_main_df.loc[len(scp_main_df)] = info



In [ ]:
scp_main_df.head

In [ ]:
scp_main_df.to_csv('scp6999.csv')

In [ ]:
pd.read_csv('scp6999.csv', delimiter=',', quotechar='"')